# Introduction

# Imortant Libraries

In [49]:
! pip install contractions
! pip install emoji
! pip install arabic_reshaper python-bidi nltk snowballstemmer

In [50]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
import plotly.express as px

In [51]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
import emoji
import arabic_reshaper
from bidi.algorithm import get_display
from snowballstemmer import stemmer
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

In [52]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load Dataset

In [53]:
# read data
dialects_data = pd.read_csv('/kaggle/input/dialects-db/dialects_data.csv', engine='python')

In [54]:
# show the first five rows of the dataframe
dialects_data.head()

,id,text,dialect
0,1009754958479151232,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.,LY
1,1009794751548313600,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ليا انا ميليشياوي زمان وتوة,LY
2,1019989115490787200,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند,LY
3,1035479791758135168,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم هو انسان قليل الادب اصلاً. ثانياً شن ذنب يلي معندهش اب ولا ام ولا خوت ولا خوات؟ يعني اليتيمة متستحقش تتزوج؟ وثالثاً ليش البنت هي بس لازم ادير الف حساب للراجل؟ هي متستحقش يندارلها الف حساب ولا هي عبدة؟,LY
4,1035481122921164800,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتيمة ولا بنت معندهش خوت. هدي اعصابك وفكينا من التخلف امتاعك,LY


# EDA and Preprocessing


In [55]:
dialects_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147734 entries, 0 to 147733
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       147734 non-null  object
 1   text     147732 non-null  object
 2   dialect  147718 non-null  object
dtypes: object(3)
memory usage: 3.4+ MB


## check NaNs

In [56]:
dialects_data.isna().sum()

id          0
text        2
dialect    16
dtype: int64

**we will delete rows with missing values**

In [57]:
# drop rows with missing values
dialects_data = dialects_data.dropna()

In [58]:
#check for missing values ag
dialects_data.isna().sum()

id         0
text       0
dialect    0
dtype: int64

## check duplicates
we need drop id column before check duplicates

In [59]:
# Drop the 'id' column
dialects_data = dialects_data.drop(columns=['id'])

In [60]:
dialects_data.head()

,text,dialect
0,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.,LY
1,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ليا انا ميليشياوي زمان وتوة,LY
2,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند,LY
3,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم هو انسان قليل الادب اصلاً. ثانياً شن ذنب يلي معندهش اب ولا ام ولا خوت ولا خوات؟ يعني اليتيمة متستحقش تتزوج؟ وثالثاً ليش البنت هي بس لازم ادير الف حساب للراجل؟ هي متستحقش يندارلها الف حساب ولا هي عبدة؟,LY
4,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتيمة ولا بنت معندهش خوت. هدي اعصابك وفكينا من التخلف امتاعك,LY


In [61]:
# Check for duplicates
duplicates = dialects_data.duplicated()

# Print the number of duplicate rows
print(f"Number of duplicate rows: {duplicates.sum()}")

Number of duplicate rows: 0


## check dataset balancing

In [62]:
# Plot the distribution of 'dialect' column with colorful bars
fig = px.histogram(dialects_data, x='dialect', color='dialect',
                   title='Distribution of Dialects',
                   labels={'dialect': 'Dialect', 'count': 'Frequency'},
                   color_discrete_sequence=px.colors.qualitative.Set1)
fig.update_xaxes(categoryorder='total descending')
fig.update_layout(xaxis_title="Dialect", yaxis_title="Frequency")
fig.show()

/opt/conda/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



## **Ooh there is big imbalance here**
We Can deal with that in difference ways.

Here are a few strategies we can do to handle imbalanced datasets:

*    **Resampling Techniques:**
        Over-sampling: Increase the number of minority class samples by randomly duplicating them or generating synthetic samples (e.g., using SMOTE).
        Under-sampling: Decrease the number of majority class samples by randomly removing them.
        Combined over- and under-sampling: A combination of over-sampling the minority class and under-sampling the majority class.

*    **Algorithmic Techniques:**
        Use algorithms that are robust to class imbalance, such as tree-based algorithms (Random Forest, Gradient Boosting) or ensemble methods.
        Adjust class weights in the model to penalize misclassification of minority classes more.

*    **Evaluation Metrics:**
        Choose appropriate evaluation metrics that are less sensitive to class imbalance, such as precision, recall, F1-score.



## Show a representative sample of data texts to find out required cleaning steps.

In [63]:
sample_data = dialects_data.sample(10)
sample_data

,text,dialect
10122,@MohammedSafar93 @arabqoute ويارتنى مكنتش لعبت معاك،،، يلعن ابو شكلك,LY
43981,يلاه اللي ناعس يفيق واللي مزال فايق يمشي ينعس..,MA
20144,@jeedr2004 وين ام القنديل شرق ولا غرب !,LY
7610,مشكله اَي وزير عندنا ميحطش خطه حتى متوسطه الأمد ويشتغل عليها حتى لو هوا ترك الوزاره يكمل اللي بعده \nكل حلولنا وقتيه من اجل انتصارات للدعاية السياسية والوزير اللي يجي يبدا من الصفر,LY
9206,@Aliswan16 انت بالذات مفروض يحبسوك في ليبيا تطلع ادير لعار,LY
87281,امتى حنقتنع داخليا بجد ان السياسة ملهاش دعوة بالدين واقتناعنا يكون حقيقى لدرجة انه يظهر فى كلامنا.\nانا مسيحية وفخورة ان اللى بيحكمنى #السيسي ومش فارق معايا دينه\nولو اتحطيت فى اختيار بينه وبين مسيحى اقل فى الكفاءة والاخلاص والامانة اكيد حاختار #السيسي,EG
55463,@MegaFMeg @omertaher احنا عايشين عصر اﻻوفر والتربص،ولو ده هيبقي المنطق هنﻻقي اجانب اﻻخر شبهوا الرئيس بمرسي ف مش هنخلص اتهامات,EG
146938,@alimoha73879480 @OMDz6ktXClBIUos ما رديت على السؤال😁,SD
73270,هل تصبح فيفى عبده نجمة هذا الشهر ؟ https://t.co/hKFIpwuUuo,EG
19234,@AlhoneS ليه هارب ساعتين وربع بزبط,LY


## Cleaning steps that we will need are:

- Remove Tatweel
- Remove digits and symbols
- Remove emojis
- Remove URLs
- Remove usernames
- Remove non-Arabic characters
- Remove extra spaces
- Remove hashtags
- Remove un-ASCII characters
- Handling Noise and Garbage Characters
- Handling Bi-directional Text

## Split data into features and target

In [64]:
# Separate features (X) and target variable (y)
X_text = dialects_data['text']  # Text features
y = dialects_data['dialect']  # Target variable

In [65]:
label_encoder = LabelEncoder()
label_encoder.fit(y)


LabelEncoder()

In [66]:
y = label_encoder.transform(y)

In [67]:
y = pd.DataFrame(y, columns=['dialect'])

In [68]:
y.shape

(147718, 1)

In [69]:
y.value_counts()

dialect
0          57635
2          36499
1          27617
4          14433
3          11534
Name: count, dtype: int64

In [70]:
X_text.head()

0                                                                                                                                                                                          @toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.
1                                                                                                                                                                                                                                   @AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ليا انا ميليشياوي زمان وتوة
2                                                                @smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند
3    @AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون

## Split data into train,valid and test sets

In [71]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X_text, y, test_size=0.2, random_state=42)


In [72]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.25    , random_state=42)

In [73]:
X_train.shape, X_valid.shape, X_test.shape

((88630,), (29544,), (29544,))

## Define the custom transformer for Arabic text cleaning

In [74]:
class ArabicTextCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, use_stemming=False):
        self.use_stemming = use_stemming
        self.arabic_stopwords = set(stopwords.words('arabic'))
        
        if self.use_stemming:
            self.stemmer = stemmer("arabic")
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # If X is a single string, convert it to a list of one element
        if isinstance(X, str):
            X = [X]
        return [self.clean_text(text) for text in X]
    
    def clean_text(self, text):
        print("Original text:", text)

        # Removing Non-Arabic Characters
        text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
        print("After removing non-Arabic characters:", text)

        # Removing Tatweel
        text = text.replace('ـ', '')
        print("After removing Tatweel:", text)

        # Removing HTML Tags
        text = re.sub(r'<.*?>', '', text)
        print("After removing HTML tags:", text)

        # Tokenization
        words = word_tokenize(text)
        print("After tokenization:", words)

        # Removing Stopwords
        words = [word for word in words if word not in self.arabic_stopwords]
        print("After removing stopwords:", words)

        # Removing Digits and Symbols
        text = re.sub(r'[0-9’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+', '', ' '.join(words))
        print("After removing digits and symbols:", text)

        # Stemming
        if self.use_stemming:
            words = [self.stemmer.stemWord(word) for word in words]
        print("After stemming:", words)

        text = ' '.join(words)
        print("After joining words:", text)

        # Remove hashtag
        text = re.sub(r'#([^\s]+)', '', text)
        print("After removing hashtags:", text)

        # Removing URLs and Email Addresses and usernames
        text = re.sub(r'http\S+|www\S+|@\S+', '', text)
        print("After removing URLs and usernames:", text)

        # Removing Emojis and Symbols
        text = ''.join(char for char in text if not emoji.is_emoji(char))
        
        print("After removing emojis and symbols:", text)

        # Removing Repeated Characters
        text = re.sub(r'(.)\1{2,}', r'\1\1', text)
        print("After removing repeated characters:", text)

        # Handling Noise and Garbage Characters
        text = re.sub(r'[^\u0600-\u06FF\s]', '', text)  # Removing non-Arabic characters again
        print("After handling noise and garbage characters:", text)

       
        return text


In [75]:
# Try the ArabicTextCleaner class on the sample data
n="@smsm071990 @ALMOGRBE ♥️😂🥰👩🏻‍💻❎✅ كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند	"
cleaning = ArabicTextCleaner()
cleaned_n = cleaning.transform(n)
print(cleaned_n)

Original text: @smsm071990 @ALMOGRBE ♥️😂🥰👩🏻‍💻❎✅ كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب
and they live happily ever after
ذي اند	
After removing non-Arabic characters:    كل  تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب
     
ذي اند	
After removing Tatweel:    كل  تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب
     
ذي اند	
After removing HTML tags:    كل  تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب
     
ذي اند	
After tokenization: ['كل', 'تانيه', 'شاب', 'ليبي', 'بيرتاح', 'لبنت', 'مختلفة', 'ويلاحظ', 

In [76]:
class ArabicTextCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, use_stemming=False):
        self.use_stemming = use_stemming
        self.arabic_stopwords = set(stopwords.words('arabic'))
        
        if self.use_stemming:
            self.stemmer = stemmer("arabic")
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # If X is a single string, convert it to a list of one element
        if isinstance(X, str):
            X = [X]
        return [self.clean_text(text) for text in X]
    
    def clean_text(self, text):
        
        # Removing Non-Arabic Characters
        text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
        

        # Removing Tatweel
        text = text.replace('ـ', '')

        # Removing HTML Tags
        text = re.sub(r'<.*?>', '', text)

        # Tokenization
        words = word_tokenize(text)

      

        # Removing Digits and Symbols
        text = re.sub(r'[0-9’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+', '', ' '.join(words))

        # Stemming
        if self.use_stemming:
            words = [self.stemmer.stemWord(word) for word in words]

        text = ' '.join(words)


        # Remove hashtag
        text = re.sub(r'#([^\s]+)', '', text)

        # Removing URLs and Email Addresses and usernames
        text = re.sub(r'http\S+|www\S+|@\S+', '', text)
    

        # Removing Emojis and Symbols
        text = ''.join(char for char in text if not emoji.is_emoji(char))

        # Removing Repeated Characters
        text = re.sub(r'(.)\1{2,}', r'\1\1', text)
        
        # Handling Noise and Garbage Characters
        text = re.sub(r'[^\u0600-\u06FF\s]', '', text)  # Removing non-Arabic characters again

        return text

In [77]:
cleaning = ArabicTextCleaner()
cleaned_x_train = cleaning.transform(X_train)
cleaned_x_train[0]

'اي خبر او تطور من جل الديب حدا يدقلي ما بفتحلو خط ح اغفى شوي'

In [78]:
cleaned_x_valid = cleaning.transform(X_valid)
cleaned_x_test = cleaning.transform(X_test)

# Save the cleaned data to CSV files to use it later in modeling.

In [79]:
# Combine the transformed data and labels into DataFrames
df_cleaned_x_test = pd.DataFrame({'cleaned_x_test': cleaned_x_test, 'y_test':y_test.squeeze()})
df_cleaned_x_train = pd.DataFrame({'cleaned_x_train': cleaned_x_train, 'y_train':y_train.squeeze()})
df_cleaned_x_valid = pd.DataFrame({'cleaned_x_valid': cleaned_x_valid, 'y_valid':y_valid.squeeze()})

In [80]:
# Save the DataFrames to CSV files
df_cleaned_x_train.to_csv('cleaned_x_train.csv', index=False)
df_cleaned_x_valid.to_csv('cleaned_x_valid.csv', index=False)
df_cleaned_x_test.to_csv('cleaned_x_test.csv', index=False)

# Tokenization

In [81]:
class CustomTokenizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        
        self.tokenizer = Tokenizer()

    def fit(self, X, y=None):
        # Fit the tokenizer on the input text
        self.tokenizer.fit_on_texts(X)
        return self

    def transform(self, X, y=None):
       
        # Tokenize the input text into sequences of tokens
        sequences = self.tokenizer.texts_to_sequences(X)
        return sequences

In [82]:
# Instantiate an object of the CustomTokenizer class
tokenizer = CustomTokenizer()

# Fit the tokenizer on the training data
xtrain_tokens=tokenizer.fit(cleaned_x_train )

x_train_idx = tokenizer.transform(cleaned_x_train)
x_train_idx[0]

[53, 598, 40, 4565, 1, 9495, 7404, 111, 43667, 3, 68138, 1099, 325, 43668, 293]

In [83]:
x_valid_idx = tokenizer.transform(cleaned_x_valid)
x_test_idx = tokenizer.transform(cleaned_x_test)

# Calculate max lenth for sentence and determine VOCAB SIZE

In [84]:
max_sequence_len = 0
for sentence in X_train:
    max_sequence_len = max(len(sentence), max_sequence_len)
print(max_sequence_len)

698


In [85]:
import plotly.express as px

# Calculate lengths of sentences in x_train
sentence_lengths = [len(sentence) for sentence in x_train_idx]

# Create histogram figure using Plotly Express
fig = px.histogram(x=sentence_lengths, title='Distribution of Sentence Lengths in x_train_idx',
                   labels={'x': 'Sentence Length', 'y': 'Frequency'})
                  
fig.show()

According to to the Distribution of Sentence Lengths in x_train we will
set **max_length to 50**.

In [86]:
max_sequence_len = 50

In [87]:
# Extract unique tokens from the tokenized sequences
unique_tokens = set(token for sequence in x_train_idx for token in sequence)

In [88]:
# Calculate the vocabulary size based on the number of unique tokens
VOCAB_SIZE = len(unique_tokens)
print("Vocabulary Size:", VOCAB_SIZE)

Vocabulary Size: 187676


# Padding
**Pad sequences to ensure uniform input size for the RNN. This step is crucial because RNNs require input sequences of the same length.**

In [89]:
class padding(BaseEstimator, TransformerMixin):
    def __init__(self, max_sequence_len):

        self.max_sequence_len = max_sequence_len

    def fit(self, X, y=None):

        return self

    def transform(self, X):

        # Initialize the padded dataset with zeros
        X_padded = np.zeros((len(X), max_sequence_len))

        # Pad each sequence
        for i, sent in enumerate(X):
            X_padded[i, :min(len(sent), self.max_sequence_len)] = sent[:self.max_sequence_len]

        return X_padded

In [90]:
pt=padding(max_sequence_len)
x_train_padded =pt.fit_transform(x_train_idx)
x_valid_padded =pt.fit_transform(x_valid_idx)
x_test_padded =pt.fit_transform(x_test_idx)

In [91]:
x_train_padded[0]

array([5.3000e+01, 5.9800e+02, 4.0000e+01, 4.5650e+03, 1.0000e+00,
       9.4950e+03, 7.4040e+03, 1.1100e+02, 4.3667e+04, 3.0000e+00,
       6.8138e+04, 1.0990e+03, 3.2500e+02, 4.3668e+04, 2.9300e+02,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00])

# Save the padded data to CSV files to use it later in modeling.

In [92]:
# Create DataFrames from the padded sequences
df_x_train_padded = pd.DataFrame(x_train_padded)
df_x_valid_padded = pd.DataFrame(x_valid_padded)
df_x_test_padded = pd.DataFrame(x_test_padded)



In [93]:
df_x_test_padded.head(20)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,20.0,21.0,1572.0,617.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,955.0,163.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,32.0,6521.0,644.0,2.0,89283.0,78.0,469.0,282.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,749.0,33380.0,578.0,1857.0,190.0,73.0,18.0,19.0,7815.0,341.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30.0,6.0,674.0,13006.0,692.0,875.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,20150.0,590.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7.0,4764.0,351.0,1023.0,133.0,37812.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,25.0,19.0,825.0,19994.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,3383.0,14147.0,2597.0,531.0,31491.0,263.0,7277.0,2.0,626.0,2278.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,447.0,21.0,121181.0,4.0,25311.0,22444.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [94]:
# Add the labels to the DataFrames
df_x_train_padded['y_train'] = y_train.values
df_x_valid_padded['y_valid'] = y_valid.values
df_x_test_padded['y_test'] = y_test.values


In [95]:
df_x_train_padded.head(20)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,y_train
0,53.0,598.0,40.0,4565.0,1.0,9495.0,7404.0,111.0,43667.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,564.0,644.0,26071.0,6.0,1895.0,3113.0,12139.0,8884.0,1058.0,125.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,5775.0,413.0,5775.0,5775.0,5775.0,5775.0,5775.0,5775.0,101.0,5775.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,99.0,3.0,10234.0,8318.0,1390.0,2.0,804.0,68139.0,1592.0,66.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,10.0,1694.0,5513.0,29.0,2140.0,456.0,1.0,63.0,3.0,68140.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
5,15.0,59.0,280.0,68144.0,7.0,68145.0,6658.0,1.0,21836.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
6,19.0,16.0,9496.0,68152.0,8885.0,1172.0,1059.0,26.0,16508.0,43670.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
7,68153.0,102.0,211.0,797.0,3114.0,12141.0,265.0,298.0,6047.0,12142.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8,569.0,43672.0,136.0,3285.0,68154.0,1839.0,3286.0,68155.0,2213.0,143.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
9,43673.0,752.0,13278.0,3759.0,278.0,1630.0,172.0,34.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [97]:
# Save the DataFrames to CSV files
df_x_train_padded.to_csv('x_train_padded.csv', index=False)
df_x_valid_padded.to_csv('x_valid_padded.csv', index=False)
df_x_test_padded.to_csv('x_test_padded.csv', index=False)


----